# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[student1@ulb.ac.be](mailto:student1@ulb.ac.be) - Student ID 123456__
### Student 2 - __[student2@vub.ac.be](mailto:student2@ulb.ac.be) - Student ID 789012__
### Student 3 - __[student3@ulb.ac.be](mailto:student3@ulb.ac.be) - Student ID 345678__

### Video presentation: www.youtube.com/abcd1234

## Project Title


# Introduction


# Data preprocessing

In [4]:
library(rlang)
library(dummies)
library(stringdist)
library(dplyr)
library(randomForest)

dummies-1.5.6 provided by Decision Patterns


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine



In [5]:
df <- read.csv("training_data.csv", stringsAsFactors=T, header = TRUE, strip.white = TRUE, sep = ",")
#df <- df[, setdiff(names(df), "X")]
#df_labels <- read.csv("training_labels.csv", header = TRUE, strip.white = TRUE, sep = ",")

In [6]:
lowercase_all = function(df) {
    for(column in names(df)) {
        columnClass = class(df[,column])
        if(columnClass == "factor") {
            df[,column] <<- tolower(df[,column])
        }
    }
}

In [7]:
lowercase_all(df)

In [8]:
#filter long/lat
x_up_right = 0.022
y_up_right = 40.729
x_down_left = -12.729
y_down_left = 28.138

df <- df[df$latitude > x_down_left & df$latitude < x_up_right & df$longitude > y_down_left & df$longitude < y_up_right, ] #filter pumps in atlantic fucking ocean

## Missing value imputation

In [9]:
work_on = setdiff(names(df), c("id"))
df[work_on][df[work_on] == 0 | df[work_on] == "-" | df[work_on] == "" | df[work_on] == "unknown"] = NA

In [10]:
drops <- c("latitude", "longitude", "funder", "date_recorded", "gps_height", "wpt_name", "num_private", "lga", "ward", "subvillage", "region_code", "district_code", "recorded_by", "scheme_name", "waterpoint_type_group", "payment", "management", "management_group", "extraction_type_group", "extraction_type", "quantity_group", "quality_group", "amount_tsh", "population", "public_meeting", "scheme_management", "permit", "source_type", "source_class")
df = df[ , !(names(df) %in% drops)] #drop useless (a priori) features, maybe after feature selection remove less

In [11]:
similarity_test_apply = function(line) {
    NAs = is.na(line)
    idxs = which(is.na(line))
            #print(df[names(df)[idxs]])
    line2 = line[-idxs]
    most_similar = full_df[which.min(Reduce(`+`,Map(stringdist, setdiff(line2, full_df[, -idxs]), line2, method='jaccard'))),]

    line[idxs] <- most_similar[idxs] 
    to_change = which(strtoi(df$id) == strtoi(line$id))
    assign('df',df,envir=.GlobalEnv)
    df[to_change, idxs] <<- most_similar[idxs]
        
}

In [14]:
sum(complete.cases(df)) #check how many rows contain at least one NA val
full_df <- na.omit(df)
dim(full_df)

[1] 31383

[1] 31383    11

In [13]:
na_df = dplyr::setdiff(df,full_df)
dim(na_df)

[1] 26205    11

In [16]:
#testing similarity

start.time = Sys.time()
res = apply(na_df[1:300,], 1, similarity_test_apply)
round(Sys.time() - start.time,5)

Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le plus long n'est pas d'une longueur multiple du plus court”Warning message in mapply(FUN = f, ..., SIMPLIFY = FALSE):
“l'argument le pl

Time difference of 6.02283 mins

## Feature engineering

In [17]:
df_labels <- read.csv("training_labels.csv", stringsAsFactors=T, header = TRUE, strip.white = TRUE, sep = ",")

In [18]:
indx <- tail(names(sort(table(df$installer))),10)
print(indx)
to_change = which(!(full_df$installer %in% indx))
full_df[to_change, 'installer'] = 'other'
full_df

 [1] "tcrs"               "central government" "kkkt"              
 [4] "district council"   "danida"             "commu"             
 [7] "rwe"                "hesawa"             "government"        
[10] "dwe"               


,id,installer,basin,region,construction_year,extraction_type_class,payment_type,water_quality,quantity,source,waterpoint_type
1,69572,other,lake nyasa,iringa,1999,gravity,annually,soft,enough,spring,communal standpipe
2,8776,other,lake victoria,mara,2010,gravity,never pay,soft,insufficient,rainwater harvesting,communal standpipe
3,34310,other,pangani,manyara,2009,gravity,per bucket,soft,enough,dam,communal standpipe multiple
4,67743,other,ruvuma / southern coast,mtwara,1986,submersible,never pay,soft,dry,machine dbh,communal standpipe multiple
6,9944,dwe,pangani,tanga,2009,submersible,per bucket,salty,enough,other,communal standpipe multiple
11,49056,other,wami / ruvu,pwani,2011,submersible,never pay,salty,enough,machine dbh,other
12,50409,danida,lake nyasa,ruvuma,1987,handpump,on failure,soft,insufficient,shallow well,hand pump
14,50495,other,pangani,kilimanjaro,2009,gravity,monthly,soft,enough,spring,communal standpipe
16,61848,dwe,lake tanganyika,rukwa,1991,handpump,never pay,soft,enough,machine dbh,hand pump
17,48451,dwe,rufiji,iringa,1978,gravity,monthly,soft,dry,river,communal standpipe


In [3]:
"(indx <- tail(names(sort(table(full_df$extraction_type_class))),17)
print(indx)
to_change = which(!(full_df$extraction_type %in% indx))
full_df[to_change, 'extraction_type'] = 'other'
full_df)"

[1] "(indx <- tail(names(sort(table(full_df$extraction_type_class))),17)\nprint(indx)\nto_change = which(!(full_df$extraction_type %in% indx))\nfull_df[to_change, 'extraction_type'] = 'other'\nfull_df)"

In [19]:
Merged_df = full_df

In [20]:
Merged_df<-merge(Merged_df,df_labels,by="id",all=FALSE)
dim(Merged_df)
Merged_df = Merged_df[,2:ncol(Merged_df)]
dim(Merged_df)

[1] 31383    12

[1] 31383    11

In [21]:
Merged_df = dummy.data.frame(Merged_df, sep="_")

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument 

In [23]:
dim(Merged_df)
Merged_df[,70:80]

[1] 31383    80

source_spring,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group_functional,status_group_functional needs repair,status_group_non functional
0,0,0,0,0,1,0,0,1,0,0
1,0,1,0,0,0,0,0,1,0,0
0,0,1,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,0,1,0
0,0,0,0,0,0,0,1,0,0,1
1,0,1,0,0,0,0,0,0,0,1
0,0,1,0,0,0,0,0,1,0,0


## Feature selection

In [ ]:
#df.pca <- prcomp(dftest_, center = TRUE,scale. = TRUE)
#summary(df.pca)

# Model selection

## Model 1

## Model 2

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


## Model 1 : Random Forest

In [24]:
#Target var not one hot encod - problem :( 3 one hot encod target features created - only one taken into consideration)
library("randomForest")

n_trees <- 20
accuracy_vec <- array(0,n_trees)

df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]
    
model_rf <- randomForest(x=train_data[,-c(target_variable)],
                         y=as.factor(train_data[,c(target_variable)]),
                         xtest=test_data[,-c(target_variable)],
                         ytest=as.factor(test_data[,c(target_variable)]),
                         ntree=n_trees, do.trace=T)
    

accuracy_vec[0] = (model_rf$test$confusion[1,1]+model_rf$test$confusion[2,2])/sum(model_rf$test$confusion)

#plot(accuracy_vec,main = "Number of trees influence",xlab = "Nbr of trees",ylab = "Classification rate") 

ntree      OOB      1      2|    Test      1      2
    1:   3.15%  1.74%  5.85%|   3.20%  1.51%  6.32%
    2:   3.04%  2.22%  4.61%|   3.00%  1.34%  6.07%
    3:   3.88%  2.42%  6.70%|   1.40%  0.73%  2.65%
    4:   4.20%  2.61%  7.28%|   1.50%  0.61%  3.16%
    5:   4.55%  2.54%  8.51%|   0.84%  0.49%  1.49%
    6:   4.26%  2.44%  7.84%|   1.22%  0.61%  2.36%
    7:   4.16%  2.24%  7.91%|   0.99%  0.59%  1.74%
    8:   3.76%  1.89%  7.43%|   1.20%  0.37%  2.72%
    9:   3.17%  1.55%  6.35%|   0.61%  0.33%  1.13%
   10:   2.61%  1.35%  5.09%|   0.61%  0.24%  1.31%
   11:   2.34%  1.14%  4.70%|   0.40%  0.22%  0.73%
   12:   1.92%  0.95%  3.82%|   0.25%  0.10%  0.54%
   13:   1.69%  0.77%  3.50%|   0.27%  0.12%  0.54%
   14:   1.54%  0.63%  3.33%|   0.23%  0.08%  0.51%
   15:   1.23%  0.52%  2.62%|   0.19%  0.08%  0.40%
   16:   1.07%  0.49%  2.22%|   0.18%  0.04%  0.44%
   17:   1.03%  0.44%  2.18%|   0.17%  0.06%  0.36%
   18:   0.91%  0.43%  1.87%|   0.14%  0.04%  0.33%
   19:   0.8

In [25]:
y_pred_rf = predict(model_rf, newdata = test_data[,-target_variable])

ERROR: Error in predict.randomForest(model_rf, newdata = test_data[, -target_variable]): No forest component in the object


In [26]:
print((model_rf$test$confusion[1,1]+model_rf$test$confusion[2,2])/sum(model_rf$test$confusion))

[1] 0.9985976


## Model 2 : Neural Network

In [27]:
library(nnet)

df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]

hidden_nodes<-72
threshold<-0.5
target_variable <-c(ncol(Merged_df)-2,ncol(Merged_df)-1,ncol(Merged_df))

model_nn<-nnet(x=train_data[,-target_variable],
               y=train_data[,target_variable],
               size=hidden_nodes,
               softmax=T,
               skip=FALSE,
               trace=T, 
               maxit=1000,
               MaxNWts=100000,
               rang=0.5)

model_nn

Y_pred<-predict(model_nn,test_data[,-target_variable])




#Y_hat_one_hot <- ifelse(Y_pred[,2] > threshold,"functional","functional needs repair")

head(Y_pred)

# weights:  5835
initial  value 42805.787271 
iter  10 value 20332.990796
iter  10 value 20332.990793
iter  10 value 20332.990793
final  value 20332.990793 
converged


a 77-72-3 network with 5835 weights
options were - softmax modelling 

,status_group_functional,status_group_functional needs repair,status_group_non functional
2636,0.5875005,0.06891277,0.3435867
17165,0.5875005,0.06891277,0.3435867
20360,0.5875005,0.06891277,0.3435867
27280,0.5875005,0.06891277,0.3435867
4994,0.5875005,0.06891277,0.3435867
13457,0.5875005,0.06891277,0.3435867


In [28]:
y_hat_idx = cbind(1:nrow(Y_pred), max.col(Y_pred, 'first'))

In [29]:
Y <- test_data[,target_variable]
y_idx = cbind(1:nrow(Y), max.col(Y, 'first'))

In [30]:
good_preds = which(y_hat_idx[,2] == y_idx[,2])
n_good = length(good_preds)
print(n_good/nrow(Y))

[1] 0.6036197


## Model 3 :  Support Vector Machine

In [34]:
#install.packages("e1071")
library(e1071)

df_idx <- sample(1:nrow(Merged_df))
half_split <- floor(nrow(Merged_df)/4*3)
target_variable1 <-ncol(Merged_df)-2
target_variable2 <-ncol(Merged_df)-1 
target_variable3 <-ncol(Merged_df)


    #3.1 Take the first half of the dataset as a training data set
train_data <- Merged_df[df_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
test_data <- Merged_df[df_idx[(half_split+1):nrow(Merged_df)],]

train_funct = train_data[,-which(names(train_data) == "status_group_functional needs repair" | names(train_data) == "status_group_non functional")]
train_funct_rep = train_data[,-which(names(train_data) == "status_group_functional" | names(train_data) == "status_group_non functional")]
train_non_funct = train_data[,-which(names(train_data) == "status_group_functional needs repair" | names(train_data) == "status_group_functional")]

test_funct = test_data[,-which(names(test_data) == "status_group_functional needs repair" | names(train_data) == "status_group_non functional")]
test_funct_rep = test_data[,-which(names(test_data) == "status_group_functional" | names(train_data) == "status_group_non functional")]
test_non_funct = test_data[,-which(names(test_data) == "status_group_functional needs repair" | names(train_data) == "status_group_functional")]

DS<-cbind(train_funct,functional = train_funct[,ncol(train_funct)])

# Model fit (using lm function)
model_svm_f<- svm(functional~.,DS)

# Model prediction 
Y_hat_f<- predict(model_svm_f,test_funct)

DS<-cbind(train_funct_rep,funct_rep = train_funct_rep[,ncol(train_funct_rep)])

# Model fit (using lm function)
model_svm_fr<- svm(funct_rep~.,DS)

# Model prediction 
Y_hat_fr<- predict(model_svm_fr,test_funct_rep)

DS<-cbind(train_non_funct,non_functional = train_non_funct[,ncol(train_non_funct)])

# Model fit (using lm function)
model_svm_nf<- svm(non_functional~.,DS)

# Model prediction 
Y_hat_nf<- predict(model_svm_nf,test_non_funct)



ERROR: Error in library(e1071): there is no package called ‘e1071’


In [38]:
Y_hat_fr = data.frame(Y_hat_fr)
Y_hat_nf = data.frame(Y_hat_nf)
Y_hat_f = data.frame(Y_hat_f)
test = cbind(Y_hat_fr, Y_hat_nf, Y_hat_f)
test

ERROR: Error in data.frame(Y_hat_fr): objet 'Y_hat_fr' introuvable


In [ ]:
y_hat_idx_svm = cbind(1:nrow(test), max.col(test, 'first'))
y_hat_idx_svm

In [ ]:
Y <- test_data[,ncol(test_data)-2:ncol(test_data)]
Y

In [ ]:
good_preds = which(y_hat_idx[,2] == y_idx[,2])
n_good = length(good_preds)
print(n_good/nrow(Y))

In [ ]:
colnames(Y_hat_f) <- c("funct")
colnames(Y_hat_nf) <- c("non functional")
colnames(Y_hat_fr) <- c("funct needs rep")

In [ ]:
test_funct

In [ ]:
Y_hat_fr

# Alternative models





# Conclusions